# Тестовое задание

### Содержание

1. [Подготовка](#1)
1. [База данных](#2)
    * [Основные таблицы](#2.1)
    * [Логирование изменений](#2.2)
1. [Задание с доступом к DWH](#3)
1. [Когда доступа нет](#4)


### Подготовка <a id = "1"> </a>


In [19]:
# установим необходимые библиотеки
import warnings
warnings.filterwarnings('ignore')
!pip install pandas
!pip install numpy
!pip install ipython-sql
!pip install Faker

In [2]:

# библиотеки
import pandas as pd
import numpy as np
from faker import Faker
import sqlite3
import datetime 

In [3]:

# Подключаемся к базе данных SQLite через магическую команду %sql
%load_ext sql
%sql sqlite:///test.db
%sql PRAGMA foreign_keys = ON

conn = sqlite3.connect('test.db')
cursor = conn.cursor()


 * sqlite:///test.db
Done.


### База данных <a id = "2"></a>


#### Основные таблицы <a id = "2.1"></a>


    Сама база может быть куда обширнее и больше, но так как нам нужна информация по вкладам, то и таблиц нам понадобится три. Две очевидны, это база с данными клиентов и база с данными по вкладам. Последняя таблица будет с данными об изменениях данных пользователей, но об этом чуть позже.

С таблицей клиентов и таблицей вкладов, схема будет выглядеть примерно так

![Image Description](img/ERD.png)



Пара прояснений по поводу таблицы вкладов

* interest_rate - процентная ставка
* start_date - дата открытия
* term - срок вклада
* maturity_date - дата ожидаемого закрытия
* closure_date - дата реального закрытия

Создадим таблицы и наполним их случайными данными.

In [4]:
%%sql
drop table clients;
drop table deposits;

 * sqlite:///test.db
Done.
Done.


[]

In [5]:
# таблица clients
cursor.execute("""
    create table if not exists clients(
    client_id INTEGER PRIMARY KEY AUTOINCREMENT,
    first_name TEXT ,
    last_name TEXT ,
    middle_name TEXT,
    address TEXT,
    email TEXT,
    phone_number TEXT 
    )""")
# таблица deposits
cursor.execute("""
               create table if not exists  deposits(
    deposit_id INTEGER PRIMARY KEY AUTOINCREMENT,
    client_id INTEGER,
    interest_rate REAL,
    start_date DATE,
    term INTEGER,
    maturity_date DATE,
    closure_date DATE,
    FOREIGN KEY (client_id) REFERENCES customers(client_id) ON DELETE CASCADE
    )""")
# очистим таблицы если они не пустые
cursor.execute("""
               delete from clients""")
cursor.execute("""
                delete from deposits""")


In [6]:
def get_client_data(n):
    faker = Faker("ru-RU")
    return [
        (
            faker.first_name(),
            faker.last_name(),
            faker.middle_name(),
            faker.address(),
            faker.email(),
            faker.phone_number(),
        )
        for _ in range(n)
    ]
n = 5000
data = get_client_data(n)
cursor.executemany(
    "INSERT INTO clients (first_name, last_name, middle_name, address, email, phone_number) VALUES (?, ?, ?, ?, ?, ?)",
    data,
)
conn.commit()

In [7]:
client_ids = %sql select client_id from clients
%sql select * from clients order by random() limit 5

 * sqlite:///test.db
Done.
 * sqlite:///test.db
Done.


client_id,first_name,last_name,middle_name,address,email,phone_number
1129,Регина,Фролова,Гурьевич,"г. Избербаш, пер. Кубанский, д. 3/4, 736958",trifon1974@example.com,8 136 707 15 38
3101,Октябрина,Пономарев,Аскольдовна,"г. Торжок, пр. Крупской, д. 913, 213263",stojan_2008@example.com,+72867619578
4150,Савва,Виноградова,Геннадьевна,"клх Карталы, ул. Спартака, д. 7, 506794",leonidpahomov@example.net,8 747 240 2599
3762,Родион,Большаков,Васильевич,"к. Грозный, ш. Маяковского, д. 236, 366122",ignatevadam@example.com,8 804 260 1504
3138,Нестор,Лазарева,Ефремович,"с. Кисловодск, ш. Спортивное, д. 9 к. 39, 204143",venedikt2020@example.com,8 598 805 44 43


In [8]:
client_ids = np.array(client_ids).reshape(-1)

# предположим что половина клиентов имеют вклад в банке
dep_clients = np.random.choice(client_ids, int(0.5*n), replace=False)
# генерация данных вкладов
def get_deposit_data(dep_clients):
    faker = Faker("ru-RU")
    result = []
    for client in dep_clients:
        interest_rate = np.round(np.random.uniform(0.05, 0.2), 2)
        term = np.random.choice([6, 9, 12, 24, 36])
        start_date = faker.date_between(start_date="-2y", end_date="-1y")
        maturity_date = start_date + datetime.timedelta(days=int(30*term))
        closure_date = maturity_date if maturity_date < datetime.date.today() else np.random.choice([None, faker.date_between(start_date="-1m", end_date="today")])
        result.append(
            (
                str(client),
                interest_rate,
                start_date,
                str(term),
                maturity_date,
                closure_date,
            )
        )

    return result

deposit_data = get_deposit_data(dep_clients)
cursor.executemany(
    "insert into deposits (client_id, interest_rate,start_date, term, maturity_date, closure_date) values (?, ?, ?, ?, ?, ?)",
    deposit_data,
)
conn.commit()
%sql select * from deposits limit 5


 * sqlite:///test.db
Done.


deposit_id,client_id,interest_rate,start_date,term,maturity_date,closure_date
1,3070,0.19,2022-11-29,12,2023-11-24,2023-11-24
2,3506,0.12,2022-07-31,9,2023-04-27,2023-04-27
3,1141,0.08,2022-06-14,6,2022-12-11,2022-12-11
4,2267,0.1,2023-01-25,6,2023-07-24,2023-07-24
5,2573,0.06,2022-03-18,6,2022-09-14,2022-09-14


#### Логирование изменений <a id = "2.2"></a>


    Как правило, мы хоти отслеживать любые изменения в таблицах. Большинство СУБД имеют встроенные механизмы для логирования таких изменений, включая инструменты типа CDC и стандартные журналы транзакций.
    Следить за изменениями можно разными способами. Например, можно использовать дополнительные таблицы, приявзать к ним триггеры, или интегрировать логирование непосредственно в транзакции, касающиеся интересующих нас таблиц. Конечно, всегда есть возможность просто логи смотреть и трогать.

    В нашем случае, предположу, что номера телефонов меняются редко, можно использовать триггеры для операций обновления в таблице clients, записывая изменения напрямую в таблицу аудита log. 

Поэтому предлагаю создать таблицу для отслеживания изменения номеров и создать триггер. 


![Image Description](img/ERD_w_a.png)


In [9]:
# таблица логов смены номеров телефонов
cursor.execute("""
            create table if not exists  log(
                client_id INTEGER,
                old_phone TEXT,
                new_phone TEXT,
                date DATE,
                FOREIGN KEY (client_id) REFERENCES clients(client_id) ON DELETE CASCADE
                )
            """)
cursor.execute("""
               delete from log""")
# создадим триггера на изменение данных в клиенте
cursor.execute("""
    create trigger update_phone after update on clients
    begin
        insert into log (client_id, old_phone, new_phone, date) values (old.client_id, old.phone_number, new.phone_number, date('now'));
    end
    """
)
conn.commit()

In [10]:
random_phone = Faker("ru-RU").phone_number()
random_client = str(np.random.choice(client_ids))
%sql update clients set phone_number = :random_phone where client_id = :random_client
%sql select * from log

 * sqlite:///test.db
1 rows affected.
 * sqlite:///test.db
Done.


client_id,old_phone,new_phone,date
1688,8 (769) 854-1983,+73221805483,2024-03-08


Триггер работает а значит давайте сгенерируем данных по преждевременно закрытым вкладам и сменим несколько номеров

In [11]:
# количество не закрытых вкладов пока что
dep_clients_ids = %sql select client_id from deposits where closure_date is null
# пусть 30% вкладов закрылись преждевременно в прошлом месяце
dep_clients_ids = np.array(dep_clients_ids).reshape(-1)
n = int(0.3*len(dep_clients_ids))
to_close = np.random.choice(dep_clients_ids, n, replace=False)

"""
update deposits set closure_date = date('now', '-1 month') 
where deposit_id in (
    select deposit_id from deposits
    where closure_date is null 
    order by random() limit (select round(count(*)*0.3) from deposits where closure_date is null)
    )
"""
# ужасная штука из sqlalchemy. По хорошему стоит конечно писать нормальный запрос
cursor.executemany(
    "update deposits set closure_date = ? where client_id = ?",
    [(Faker("ru-RU").past_date(), str(deposit)) for deposit in to_close],
)
# и 25 % из этих вкладов были закрыты при условии смены номера
k = int(0.25*n)
to_change = np.random.choice(to_close, k, replace=False)
# поменяем номер у выбранных клиентов
cursor.executemany(
    "update clients set phone_number = ? where client_id = ?",
    [(Faker("ru-RU").phone_number(), str(client)) for client in to_change],
)
# и чуть  подредактирвуем логи, как будто это было в прошлом месяце до закрытия вклада
cursor.executemany(
    "update log set date = date('now', '-1 month') where client_id = ?",
    [(str(client),) for client in to_change],
)
conn.commit()

 * sqlite:///test.db
Done.


### Задание с доступом к данным <a id="3"></a>

>Опишите вариант решения задачи , когда у вас есть доступ к базе данных в АБС.
>Предположите , как могут хранится данные (спроектируйте таблицы)  опишите данные которое вам потребуется для формирования отчета.
>Предположите какие данные необходимо включить в отчет . Напишите  SQL исходя из ваших предположений.  


    Как было сказано выше, если у нас есть доступ к базе данных в АБС или даже напрямую в DWH то нам понадобятся логи измненией данных таблицы с клиентами, таблица клиентов и таблица вкладов само собой. Упрощённая схема осталась наверху, поэтому напишем запрос.


В голову приходит два запроса. 

1. Отфильтровать вклады закрытые в прошлом месяце, при это у которых дата закрытия не совпадает с датой предпологаемого закрытия. И потом смотреть какие из клиентов меняли номер телефона. 

2. Найти пользователей которые сменили телефон в прошлом месяце, посмотреть есть ли у них вклады и когда закрылись

Хоть первый и звучит логичнее, но сравнивать даты может быть дорого и не очень удобно, тем более по несколько раз. Так же я почти уверен что таблица с логами сменой номеров будет сильно меньше чем таблица вкладов, и можно будет сильно уменьшить число фуллсканов. Поэтому напишу оба, но я бы пользовался вторым.

Вроде как можно индексировать поля дат, но в данный момент я бы этого делать не стал.



In [12]:
%%sql 
with q1 as (
    SELECT client_id FROM log
    WHERE date >= date('now', 'start of month', '-1 month')
    AND date < date('now', 'start of month')
    )
SELECT * FROM deposits inner join q1 using(client_id) 
where closure_date>= date('now', 'start of month', '-1 month') and closure_date < maturity_date
limit 10

 * sqlite:///test.db
Done.


deposit_id,client_id,interest_rate,start_date,term,maturity_date,closure_date
79,2727,0.16,2022-11-20,24,2024-11-09,2024-02-08
80,1962,0.16,2022-07-24,24,2024-07-13,2024-02-27
281,2451,0.11,2022-08-26,24,2024-08-15,2024-03-04
289,2445,0.08,2022-10-15,36,2025-09-29,2024-02-20
333,2980,0.08,2022-12-21,36,2025-12-05,2024-02-24
425,3497,0.16,2022-12-11,36,2025-11-25,2024-03-02
474,3982,0.11,2022-08-17,24,2024-08-06,2024-02-18
489,4910,0.15,2022-06-16,36,2025-05-31,2024-02-19
506,2914,0.11,2022-04-13,24,2024-04-02,2024-02-10
626,2540,0.16,2022-04-06,36,2025-03-21,2024-02-16


Join операторы начинают сравнивать все строки по заданному полю, и это может сильно работу запроса. Мы можем альтернативно воспользоваться подзапросом, но подзапросы это не сильно исправит положение.

In [13]:
%%sql
with q1 as (
select * from deposits where client_id in (SELECT client_id FROM log
    WHERE date >= date('now', 'start of month', '-1 month')
    AND date < date('now', 'start of month')))
select * from q1 where
     closure_date>= date('now', 'start of month', '-1 month') and closure_date < maturity_date
limit 10

 * sqlite:///test.db
Done.


deposit_id,client_id,interest_rate,start_date,term,maturity_date,closure_date
79,2727,0.16,2022-11-20,24,2024-11-09,2024-02-08
80,1962,0.16,2022-07-24,24,2024-07-13,2024-02-27
281,2451,0.11,2022-08-26,24,2024-08-15,2024-03-04
289,2445,0.08,2022-10-15,36,2025-09-29,2024-02-20
333,2980,0.08,2022-12-21,36,2025-12-05,2024-02-24
425,3497,0.16,2022-12-11,36,2025-11-25,2024-03-02
474,3982,0.11,2022-08-17,24,2024-08-06,2024-02-18
489,4910,0.15,2022-06-16,36,2025-05-31,2024-02-19
506,2914,0.11,2022-04-13,24,2024-04-02,2024-02-10
626,2540,0.16,2022-04-06,36,2025-03-21,2024-02-16


Ну и данные нам нужны по клиентам поэтому дополним

In [14]:
%%sql 
result_set <<
with q1 as (
    SELECT * FROM log
    WHERE date >= date('now', 'start of month', '-1 month')
    AND date < date('now', 'start of month')
    ),
q2 as (
    select * from deposits
    where client_id in (SELECT client_id FROM q1)
    and closure_date>= date('now', 'start of month', '-1 month') 
    and closure_date < maturity_date 
)
select * 
from clients inner join q2 using(client_id)
inner join q1 using(client_id) 
where closure_date>= date('now', 'start of month', '-1 month') and closure_date < maturity_date


 * sqlite:///test.db
Done.
Returning data to local variable result_set


In [15]:
result_set.DataFrame().head(10)

,client_id,first_name,last_name,middle_name,address,email,phone_number,deposit_id,interest_rate,start_date,term,maturity_date,closure_date,old_phone,new_phone,date
0,2727,Юлиан,Захаров,Герасимович,"ст. Вуктыл, бул. Кузнецова, д. 35 стр. 254, 65...",doroninelizar@example.org,8 911 665 5885,79,0.16,2022-11-20,24,2024-11-09,2024-02-08,+7 (936) 589-93-16,8 911 665 5885,2024-02-08
1,1962,Феликс,Ермакова,Вячеславович,"клх Хатанга, пр. Станиславского, д. 2 стр. 97,...",sokrat1978@example.com,+7 (895) 007-71-01,80,0.16,2022-07-24,24,2024-07-13,2024-02-27,8 (122) 782-4647,+7 (895) 007-71-01,2024-02-08
2,2451,Нестор,Некрасов,Даниловна,"п. Тырныауз, бул. Стадионный, д. 728 стр. 825,...",timur83@example.net,+7 273 210 03 69,281,0.11,2022-08-26,24,2024-08-15,2024-03-04,+7 (764) 543-9091,+7 273 210 03 69,2024-02-08
3,2445,Степан,Блохина,Архиповна,"г. Рославль, алл. Космодемьянской, д. 1/5 к. 7...",leon_2023@example.com,+7 (010) 780-4808,289,0.08,2022-10-15,36,2025-09-29,2024-02-20,+7 (170) 345-15-54,+7 (010) 780-4808,2024-02-08
4,2980,Александр,Кулагина,Владиславовна,"с. Тамбей, бул. Садовый, д. 8, 809956",sokolovevse@example.org,+7 (253) 490-0156,333,0.08,2022-12-21,36,2025-12-05,2024-02-24,+7 721 768 4356,+7 (253) 490-0156,2024-02-08
5,3497,Артемий,Киселев,Валерианович,"к. Северо-Курильск, наб. Прибрежная, д. 6 к. 5...",mark_1996@example.net,85958854989,425,0.16,2022-12-11,36,2025-11-25,2024-03-02,8 (615) 364-89-90,85958854989,2024-02-08
6,3982,Панфил,Суворов,Витальевич,"к. Жуковский, ш. Моховое, д. 4/4 к. 4, 005216",lavrentisazonov@example.org,+7 (272) 971-27-15,474,0.11,2022-08-17,24,2024-08-06,2024-02-18,8 570 729 2978,+7 (272) 971-27-15,2024-02-08
7,4910,Ермолай,Беспалова,Фокич,"с. Надым, ш. Блюхера, д. 2/7 стр. 9/2, 336259",tihon1973@example.com,+7 807 216 1140,489,0.15,2022-06-16,36,2025-05-31,2024-02-19,+7 (875) 293-59-34,+7 807 216 1140,2024-02-08
8,2914,Николай,Григорьев,Дорофеевич,"г. Добрянка, пер. Павлика Морозова, д. 3/3 стр...",bikovkim@example.org,+7 133 538 86 07,506,0.11,2022-04-13,24,2024-04-02,2024-02-10,8 (631) 225-84-15,+7 133 538 86 07,2024-02-08
9,2540,Матвей,Полякова,Дмитриевна,"ст. Меренга, ул. Прибрежная, д. 9 стр. 471, 82...",krilovapraskovja@example.net,84093074555,626,0.16,2022-04-06,36,2025-03-21,2024-02-16,+7 (131) 560-7986,84093074555,2024-02-08


Ну и отсюда можем записать его в пандас и собирать любые статистики какие нам интересно. 

Ну и запрос для прошлого варианта

In [16]:
%%sql
with q1 as (
    select * from deposits 
    where closure_date >= date('now', 'start of month', '-1 month') 
    and closure_date < maturity_date
    ),
q2 as (
    select * from log
    where date >= date('now', 'start of month', '-1 month')
    and date < date('now', 'start of month')
    )
select * from clients  inner join q1 using(client_id) inner join q2 using(client_id) limit 5


 * sqlite:///test.db
Done.


client_id,first_name,last_name,middle_name,address,email,phone_number,deposit_id,interest_rate,start_date,term,maturity_date,closure_date,old_phone,new_phone,date
2727,Юлиан,Захаров,Герасимович,"ст. Вуктыл, бул. Кузнецова, д. 35 стр. 254, 656981",doroninelizar@example.org,8 911 665 5885,79,0.16,2022-11-20,24,2024-11-09,2024-02-08,+7 (936) 589-93-16,8 911 665 5885,2024-02-08
1962,Феликс,Ермакова,Вячеславович,"клх Хатанга, пр. Станиславского, д. 2 стр. 97, 857551",sokrat1978@example.com,+7 (895) 007-71-01,80,0.16,2022-07-24,24,2024-07-13,2024-02-27,8 (122) 782-4647,+7 (895) 007-71-01,2024-02-08
2451,Нестор,Некрасов,Даниловна,"п. Тырныауз, бул. Стадионный, д. 728 стр. 825, 187461",timur83@example.net,+7 273 210 03 69,281,0.11,2022-08-26,24,2024-08-15,2024-03-04,+7 (764) 543-9091,+7 273 210 03 69,2024-02-08
2445,Степан,Блохина,Архиповна,"г. Рославль, алл. Космодемьянской, д. 1/5 к. 701, 885558",leon_2023@example.com,+7 (010) 780-4808,289,0.08,2022-10-15,36,2025-09-29,2024-02-20,+7 (170) 345-15-54,+7 (010) 780-4808,2024-02-08
2980,Александр,Кулагина,Владиславовна,"с. Тамбей, бул. Садовый, д. 8, 809956",sokolovevse@example.org,+7 (253) 490-0156,333,0.08,2022-12-21,36,2025-12-05,2024-02-24,+7 721 768 4356,+7 (253) 490-0156,2024-02-08


### Задание без доступом к данным <a id="4"></a>

Ну первым шагом будет определение какие данные мне вообще нужны для создания отчёта. 
Поэтому я посмотрю на схему данных, или на карту данных если такая имеется. 

Если у меня не будет доступа к данным в первую очередь я пойду к начальнику и уточню куда мне стоит обратиться чтобы получить доступ, или где я могу взять данные прямо сейчас если отчёт нужен срочно. 

Если вдруг начальник занят, то обращусь в опытным коллегам.

Если уже всё складывается так что ответа нет, то можно написать в IT отдел и описать суть проблемы, спросить доступ ну или данные.
На этом этапе скорее всего потребуется конкретика, какие данные зачем и почему. Как обычно 10 кругов ада с доступом к sensetive data поэтому надо будет заморочиться немного и подробно описать всё. 

А как данные будет на компьютере, ну или доступ, тут уже схема отработанная:) 

Либо загрузить их в pandas обработать, либо загрузить их в excel и там сделать отчёт. 



##  Спасибо за внимание! 